# Visualisation and Interactivity improvements
## Objectives
1. ~Complete write function that takes ODC xarray output and converts it to RGBA colormap whilst accounting for single bands~
2. ~Modify `mk_image_overlay` to also accept RGB(A) values~
3. Integrate reprojection to web mercator in parent function
4. Normalising dependent on input, no data, categories, bitmask, boolean
5. Sidecar / ipyleaflet auto update instead of creating new tabs
6. Transparency slider
7. ~Basemap plotting options~
8. Options for time slices
9. Display time of image on basemap

In [1]:
import odc.algo
import odc.ui
from datacube import Datacube
from ipyleaflet import Map, ImageOverlay, basemaps
import datacube
import sys
import xarray as xr
import numpy as np
from typing import Tuple, Optional, List
from sidecar import Sidecar
from ipywidgets import IntSlider
from matplotlib import cm
import matplotlib.pyplot as plt
from IPython.display import Image, display
from matplotlib.colors import Normalize

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../Scripts')
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map

In [2]:
from matplotlib import cm
from matplotlib.colors import Normalize

def colorize(da, cmap='viridis', vmin=None, vmax=None):
    """
    Convert values in single band xarray to colormap values
    Can accept multiple time steps
    da - xarray.DataArray
    cmap - desired colormapping
    vmin / vmax - values for normalisation
    """
    normalized = Normalize(vmin=vmin, vmax=vmax)(da)
    colormapped = cm.get_cmap(cmap)(normalized, bytes=True)
    colormapped_xr = xr.DataArray(data=colormapped, 
                                  coords = da.coords, dims=da.dims+('band',))
    return colormapped_xr

In [3]:
dc = Datacube(app='viz')

In [4]:
# Load data
query = {
    'x': (153.33, 153.425),
    'y': (-27.60, -27.665),
    'time': ('2018'),
    'output_crs': 'EPSG:3857',
    'resolution': (-30, 30),
    'group_by': 'solar_day'
}


ds = dc.load(product = 'ls8_nbart_geomedian_annual', **query)

print(ds)

<xarray.Dataset>
Dimensions:  (time: 1, x: 354, y: 273)
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01
  * y        (y) float64 -3.199e+06 -3.199e+06 ... -3.207e+06 -3.207e+06
  * x        (x) float64 1.707e+07 1.707e+07 1.707e+07 ... 1.708e+07 1.708e+07
Data variables:
    blue     (time, y, x) int16 485 478 490 492 486 503 ... 239 270 273 289 298
    green    (time, y, x) int16 557 547 549 536 525 536 ... 364 408 402 413 429
    red      (time, y, x) int16 404 385 335 281 263 254 ... 318 362 344 363 381
    nir      (time, y, x) int16 166 153 128 108 109 ... 2278 2469 2269 2341 2314
    swir1    (time, y, x) int16 89 81 68 57 56 56 ... 983 1160 1115 1116 1163
    swir2    (time, y, x) int16 71 63 51 44 44 44 48 ... 495 448 544 518 528 566
Attributes:
    crs:      EPSG:3857


In [5]:
sc = Sidecar(title='Map')
m = Map(basemap=basemaps.Esri.WorldImagery, center=(query['y'][0], query['x'][0]), zoom=9)
with sc:
    display(m)

Either run colorize and pass to `mk_image_overlay` or pass a multi band dataset to it.

In [6]:
# Convert dataarray values to colormapping
cm_da = colorize(ds.red, cmap='Reds')

In [7]:
#Add to map and center
image = odc.ui.mk_image_overlay(cm_da)
# Add layer to map
m.add_layer(image)
m.center = (query['y'][0], query['x'][0])